### Simple Zero Shot - Mistral - N2C2

In [30]:
%load_ext autoreload
%autoreload 2
import os
import json
import genesis_cloud.utils as u
from tqdm import tqdm
from step.step import *

instance_url = '147.189.192.78:8080'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Loading dataset

In [31]:
dataset_dir = '../../../datasets/n2c2-2009/'
dataset_files = os.listdir(dataset_dir)
dataset = []
for file in dataset_files:
    with open(dataset_dir + file) as f:
        try:
            obj = json.load(f)
            obj['file'] = file
            dataset.append(obj)
        except:
            print('Error loading file: ', file)

## CSV

In [32]:
input_text = InputStep('input text', '')

In [33]:
prompt_dir = 'prompt_CSV.txt'
prompt = open(prompt_dir).read()

params = json.load(open('params.json', 'r'))


def map_input(inputs: list = None):
    return {'document': inputs[0].output.value}


def map_output(output):
    output = output.replace('<dummy32000>', '')
    if not output.endswith('<|im_end|>'):
        output += '<|im_end|>'
    return output


step1 = Step(LLamaCppModel(instance_url), 'step1', prompt, params, map_input, map_output, inputs=[input_text])

In [34]:
parse_csv = ParseCSVStep('parse_csv', ';', [step1])

In [35]:
ans = []
pbar = tqdm(dataset)
pipeline = Pipeline(parse_csv)
for sample in pbar:
    input_text.input.value = sample['text']
    res = pipeline()
    ans.append({'file': sample['file'], 'out': res})

100%|██████████| 100/100 [07:48<00:00,  4.69s/it]


In [35]:
import json
res = json.load(open('results_N2C2_CSV.json', 'r'))
ans = res['outputs']

In [36]:
ans = [{
    'file': file['file'],
    'out': [{
        'medication_name': out[0],
        'dosage': out[1],
        'mode': out[2],
        'frequency': out[3], 
    }
        for out in file['out'] if len(out) == 4
    ]
} for file in ans]


In [40]:
result = {
    'experiment_name': 'Simple Zero Shot - Mistral - N2C2 - CSV',
    'inputs': [file['file'] for file in dataset],
    'outputs': ans,
    'pipeline': json.loads(str(pipeline))
}
json.dump(result, open('result_CSV.json', 'w'))

### Evaluation

the dataset contains many repeated annotations, so we'll keep only the most relevant ones that include more information

In [ ]:
prompt = """
<|im_start|>user
Are these two jsons equal?

```json1
{json1}
```
```json2
{json2}
```

<|im_end|>
<|im_start|>assistant
"""

params = json.load(open('params.json', 'r'))
params['grammar'] = "root ::= 'yes' | 'no'"
def map_input(inputs: list = None):
    return {'json1': inputs[0].output.value}


def map_output(output):
    output = output.replace('<dummy32000>', '')
    if not output.endswith('<|im_end|>'):
        output += '<|im_end|>'
    return output

input = InputStep('input', '')
step1 = Step(LLamaCppModel(instance_url), 'step1', prompt, params, map_input, map_output, inputs=[])


for sample in dataset:
    annotations = sample['annotations']
    ## group by same chem_name
    groups = []
    

### JSON

In [55]:
prompt_dir = 'prompt_JSON.txt'
prompt = open(prompt_dir).read()

params = json.load(open('params.json', 'r'))


def map_input(inputs: list = None):
    return {'document': inputs[0].output.value}


def map_output(output):
    output = output.replace('<dummy32000>', '')
    if not output.endswith('<|im_end|>'):
        output += '<|im_end|>'
    return output


step1 = Step(LLamaCppModel(instance_url), 'step1', prompt, params, map_input, map_output, inputs=[input_text])

In [56]:
to_json = ToJsonStep('to json', [step1])

In [58]:
ans = []
pbar = tqdm(dataset)
pipeline = Pipeline(to_json)
for sample in pbar:
    input_text.input.value = sample['text']
    # pipeline = Pipeline(parse_csv)
    res = pipeline()
    ans.append({'file': sample['file'], 'out': res})

100%|██████████| 100/100 [08:17<00:00,  4.97s/it]


In [59]:
result = {
    'experiment_name': 'Simple Zero Shot - Mistral - N2C2 - JSON',
    'inputs': [file['file'] for file in dataset],
    'outputs': ans,
    'pipeline': json.loads(str(pipeline))
}
json.dump(result, open('result_JSON.json', 'w'))
